<h2>Install boto3</h2>

pip install boto3

Set up credentials:
* create folder in /home/user/ named ".aws" (/home/user/.aws). This folder is hidden
* create file in folder .aws with name "credentials". Paste following lines. YOUR_KEY and YOUR_SECRET (http://docs.aws.amazon.com/general/latest/gr/getting-aws-sec-creds.html)

    [default] <br/>
    aws_access_key_id = YOUR_KEY <br/>
    aws_secret_access_key = YOUR_SECRET <br/>


* create file config and paste following lines

    [default] <br/>
    region=us-east-1 <br/>


In [1]:
import boto3

import boto3
ec2 = boto3.resource('ec2')

instances = ec2.instances.filter(
    Filters=[{'Name': 'instance-state-name', 'Values': ['running']}])
instanceListNames = []
attachedVolumes = []
for instance in instances:
    print(instance.id, instance.instance_type, instance.key_name, instance.private_ip_address, instance.public_dns_name ,instance.root_device_name)
    instanceListNames.append(instance.id)
    volumeId = instance.block_device_mappings
    attachedVolumes.append(volumeId[0]['Ebs']['VolumeId'])
    print(volumeId)

    

<h3> Create budget</h3>

In [ ]:
import datetime

budget = boto3.client('budgets')

budget.create_budget(
    AccountId='310130539075',
    Budget={
        'BudgetName': 'BudgetInvestigation1',
        'BudgetLimit': {
            'Amount': '10',
            'Unit': 'USD'
        },
        'CostTypes': {
            'IncludeTax': True,
            'IncludeSubscription': True,
            'UseBlended': True
        },
        'TimeUnit': 'MONTHLY',
        'TimePeriod': {
            'Start': datetime.datetime(2018, 1, 1),
            'End': datetime.datetime(2019, 1, 1)
        },
        'CalculatedSpend': {
            'ActualSpend': {
                'Amount': '10',
                'Unit': 'USD'
            },
            'ForecastedSpend': {
                'Amount': '10',
                'Unit': 'USD'
            }
        },
        'BudgetType': 'COST'
    },
    NotificationsWithSubscribers=[
        {
            'Notification': {
                'NotificationType': 'ACTUAL',
                'ComparisonOperator': 'LESS_THAN',
                'Threshold': 90.0,
                'ThresholdType': 'PERCENTAGE'
            },
            'Subscribers': [
                {
                    'SubscriptionType': 'EMAIL',
                    'Address': 'mateusz.zbikowski@gmail.com'
                },
            ]
        },
    ]
)

<h3>Check budget</h3>

Create a budget on AWS
https://docs.aws.amazon.com/awsaccountbilling/latest/aboutv2/budgets-create.html

Check your accoutn id
https://docs.aws.amazon.com/IAM/latest/UserGuide/console_account-alias.html


In [2]:
client = boto3.client('budgets')
response = client.describe_budget(
    AccountId='xxxxxxxxxxxxxxxxxxxxx',
    BudgetName='BudgetInvestigation'
)

actualSpend = float(response['Budget']['CalculatedSpend']['ActualSpend']['Amount'])

print(actualSpend)

0.35


If limit is exceeded create a snapshot of EBS and skill all instances

In [3]:
if(actualSpend > 0):
    print('create snapshot')
    print(attachedVolumes)
    '''
    for volumeId in attachedVolumes:
        ec2.create_snapshot(VolumeId=volumeId)
    '''
    print('kill all instances')
    print(instanceListNames)
    '''
    ec2.instances.filter(InstanceIds=instanceListNames).stop()
    ec2.instances.filter(InstanceIds=instanceListNames).terminate()
    '''

create snapshot
[]
kill all instances
[]


<h3> Create lambda function</h3>

Roles IAM should be created in order to execute lambda scripts

In [11]:
lambdaFunction = boto3.client('lambda')

with open('/home/mz/DataInception/AWS/aws_monitor_instances_lambda.zip', 'rb') as content_file:
    content = content_file.read()
    
vals = {}
vals['FunctionName'] = 'terminateInstances'
vals['Role'] = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
vals['Handler'] = 'aws_monitor_instances_lambda.lambda_handler'
# vals['Code'] = {'S3Bucket': 'https://s3-ap-southeast-1.amazonaws.com/abcde/aws-lambda-image.zip'}
vals['Code'] = {'ZipFile': content}
vals['Publish'] = True
vals['Runtime'] = 'python3.6'
vals['Timeout'] = 30

lambdaFunction.create_function(**vals)



{'CodeSha256': '1YP4rb+2DlN81wFqAQIozb99bk5sF5N2rt5HGG8LIac=',
 'CodeSize': 639,
 'Description': '',
 'FunctionArn': 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxx',
 'FunctionName': 'terminateInstances',
 'Handler': 'aws_monitor_instances_lambda.lambda_handler',
 'LastModified': '2018-01-13T19:23:35.619+0000',
 'MemorySize': 128,
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
   'content-length': '601',
   'content-type': 'application/json',
   'date': 'Sat, 13 Jan 2018 19:23:35 GMT',
   'x-amzn-requestid': '4000970f-f897-11e7-8b3e-15456b468008'},
  'HTTPStatusCode': 201,
  'RequestId': '4000970f-f897-11e7-8b3e-15456b468008',
  'RetryAttempts': 0},
 'Role': 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx',
 'Runtime': 'python3.6',
 'Timeout': 30,
 'TracingConfig': {'Mode': 'PassThrough'},
 'Version': '1'}

<h3> Create alarm in the Cloud Watch</h3>

In [8]:
cloudwatch = boto3.client('cloudwatch')

# Create alarm
cloudwatch.put_metric_alarm(
    AlarmName='Budget_Monitoring',
    ComparisonOperator='GreaterThanThreshold',
    MetricName='EstimatedCharges',
    Namespace='AWS/Billing',
    Statistic='Maximum',
    Threshold=0.3,
    ActionsEnabled=False,
    Period=6,
    AlarmDescription='Alarm when bidget is exceeded',
    EvaluationPeriods=1
)

# AlarmName, AlarmDescription, ActionsEnabled, OKActions, AlarmActions, 
# InsufficientDataActions, MetricName, Namespace, Statistic, 
# ExtendedStatistic, Dimensions, Period, Unit, EvaluationPeriods, 
# Threshold, ComparisonOperator, TreatMissingData, EvaluateLowSampleCountPercentile
# Dimensions='Name=Currency,Value=USD',


{'ResponseMetadata': {'HTTPHeaders': {'content-length': '214',
   'content-type': 'text/xml',
   'date': 'Sat, 13 Jan 2018 18:30:39 GMT',
   'x-amzn-requestid': 'db93af09-f88f-11e7-95a0-6f853bc99dd1'},
  'HTTPStatusCode': 200,
  'RequestId': 'db93af09-f88f-11e7-95a0-6f853bc99dd1',
  'RetryAttempts': 0}}

<h3>Create CloudWatch Event</h3>

In [ ]:
cloudwatchEvents = boto3.client('events')

cloudwatchEvents


